** 0.0 / IMPORTS & INIT**

In [1]:
import pandas as pd
import time
import os
import pygal
import progressbar

**1.1 / CSV IMPORTING**

We import the 4 working databases from csv files in the subdirectory "Base" at the root of the current working directory through Pandas and assign them as Dataframes.                          
We also display progress information, since the process can be quite lenghty.

Caution : When all bases are imported at once, they are all loaded into RAM and they can take up to 20G. If your machine can't handle it, you should comment out what you don't need and work sequentially.

We define the columns on which we are going to work for each dataframes

In [2]:
Col_avantage = ['ligne_identifiant', 'denomination_sociale', 'categorie', 'qualite', 'benef_codepostal', 'benef_ville', 'pays', 'benef_titre_libelle', 'benef_speicalite_libelle', 'benef_etablissement_codepostal', 'ligne_type', 'avant_date_signature', 'avant_montant_ttc']
Col_convention = ['ligne_identifiant', 'denomination_sociale', 'categorie', 'qualite', 'benef_codepostal', 'benef_ville', 'pays', 'benef_titre_libelle', 'benef_speicalite_libelle', 'benef_etablissement_codepostal', 'ligne_type', 'conv_date_signature', 'conv_montant_ttc']
Col_remuneration = ['entreprise_identifiant', 'denomination_sociale', 'benef_categorie_code', 'qualite', 'benef_codepostal', 'pays', 'benef_titre_libelle', 'benef_speicalite_libelle', 'benef_etablissement_codepostal', 'remu_date', 'remu_montant_ttc']
Col_entreprise = ['pays','secteur','code_postal','ville']

In [3]:
# Imports all bases and displays progress info
# Note : last base uses a comma as separator

start = time.perf_counter() # getting starting timestamp
print('Starting import...\n\n')
print('Importing D_avantage...')
D_avantage = pd.read_csv("Base/declaration_avantage_2020_02_19_04_00.csv", sep = ";", usecols = Col_avantage)
print('D_avantage successfully imported. 3 more to go.')
print('Importing D_Convention...')
D_Convention = pd.read_csv("Base/declaration_convention_2020_02_19_04_00.csv", sep = ";", usecols = Col_convention)
print('D_Convention successfully imported. 2 more to go.')
print('Importing D_Remuneration...')
D_Remuneration = pd.read_csv("Base/declaration_remuneration_2020_02_19_04_00.csv", sep = ";", usecols = Col_remuneration)
print('D_Remuneration successfully imported. 1 more to go.')
print('Importing Entreprise...')
Entreprise = pd.read_csv("Base/entreprise_2020_02_19_04_00.csv", sep = ",", usecols = Col_entreprise)
print('Entreprise successfully imported.')
success = time.perf_counter() # getting ending timestamp
import_time = int(success - start) 
print('All csv successfully imported in %s seconds.'%(import_time))

Starting import...


Importing D_avantage...
D_avantage successfully imported. 3 more to go.
Importing D_Convention...
D_Convention successfully imported. 2 more to go.
Importing D_Remuneration...
D_Remuneration successfully imported. 1 more to go.
Importing Entreprise...
Entreprise successfully imported.
All csv successfully imported in 67 seconds.


**1.2 / DATAFRAMES CLEANING**

We aggregate all values representing less than 0.2% of the dataframe as one.

** 2.1 / PLOTTING **


In [20]:
def comparator3000():
    '''

    '''
    print('Started.')
    dic = dict()
    start = time.perf_counter()
    q = list(D_Convention.index)
    fc = 0
    sc = 0
    pc = 0
    for i in q:
        if i % 50553 == 0:
            aa = int(i / 5055300 * 100) 
            aa = str(aa)
            print('%s %% processed.'%(aa))
            
        cp = D_Convention['benef_codepostal'][i]
        ttc = D_Convention['conv_montant_ttc'][i]

        cp = str(cp)
        cp = cp[:2]
    
        try:
            cp = int(cp)
            ttc = int(ttc)
            sc += 1
        except ValueError:
            fc += 1
            continue

        if cp in dic:
            dic[cp] += ttc

        else:
            dic[cp] = ttc
            

    success = time.perf_counter()
    ns = int(success - start) 

    
    

    print('Failed %s times | Succeeded %s times in %s seconds'%(sc, fc, ns))        

    return dic


jpp = comparator3000()




Started.
0 % processed.
1 % processed.
2 % processed.
3 % processed.
4 % processed.
5 % processed.
6 % processed.
7 % processed.
8 % processed.
9 % processed.
10 % processed.
11 % processed.
12 % processed.
13 % processed.
14 % processed.
15 % processed.
16 % processed.
17 % processed.
18 % processed.
19 % processed.
20 % processed.
21 % processed.
22 % processed.
23 % processed.
24 % processed.
25 % processed.
26 % processed.
27 % processed.
28 % processed.
28 % processed.
30 % processed.
31 % processed.
32 % processed.
33 % processed.
34 % processed.
35 % processed.
36 % processed.
37 % processed.
38 % processed.
39 % processed.
40 % processed.
41 % processed.
42 % processed.
43 % processed.
44 % processed.
45 % processed.
46 % processed.
47 % processed.
48 % processed.
49 % processed.
50 % processed.
51 % processed.
52 % processed.
53 % processed.
54 % processed.
55 % processed.
56 % processed.
56 % processed.
57 % processed.
59 % processed.
60 % processed.
61 % processed.
62 % proc

In [31]:
print(jpp)
fr_chart = pygal.maps.fr.Departments(human_readable=True)
fr_chart.title = 'Conventions signées par departement'
fr_chart.add('In 2011', {76: 38845158, 13: 45853620, 34: 25953014, 35: 16340884, 56: 2470819, 17: 2775172, 33: 37502702, 69: 237761685, 90: 481767, 64: 4830634, 42: 8685178, 49: 8909801, 97: 4728458, 78: 13024738, 59: 29122002, 29: 8159572, 92: 253706764, 80: 5766949, 67: 42408295, 38: 12854691, 81: 831221, 21: 45365968, 74: 3069763, 73: 1063545, 25: 8107591, 71: 1282886, 26: 1435422, 37: 50000590, 50: 829851, 54: 10252458, 94: 56602053, 61: 2164442, 77: 3096826, 95: 5379545, 72: 2203255, 62: 5623271, 43: 207639, 86: 8614401, 75: 408476240, 70: 325453, 83: 6084317, 93: 10165438, 14: 7243162, 31: 65845923, 68: 4788187, 52: 257189, 87: 5276102, 91: 10497185, 11: 893158, 45: 3004482, 63: 13436389, 57: 2731514, 19: 718428, 65: 844697, 51: 5551415, 66: 2472870, 36: 316504, 23: 361482, 60: 5517034, 16: 762269, 44: 24744630, 28: 1736619, 24: 851813, 55: 303060, 18: 563439, 22: 2252026, 88: 600080, 30: 4738862, 10: 987235, 40: 692133, 85: 1965301, 47: 1267454, 41: 1181929, 46: 304873, 84: 3966067, 32: 340161, 89: 963335, 82: 760835, 79: 604923, 15: 234938, 58: 324291, 27: 1072675, 53: 976631, 12: 1118260, 39: 559534, 20: 1112638, 98: 1567570, 48: 61041, 2: 569513, 1: 508141, 3: 492143, 8: 241248, 6: 18620461, 7: 304306, 5: 178215, 9: 97395, 4: 98301, 0: 65499, 96: 2051, 99: 840})
print(fr_chart.render())

12 0.27302,-0.965 0.36126,-1.4458 0.40465,-0.4999 0.24834,-1.1755 0.70778,-1.6634 -0.0181,-0.3132 0.002,-0.8249 0.38848,-0.4432 0.36044,-0.1283 0.92216,0.02 0.78912,-0.568 0.21712,-0.3032 0.60473,-0.4599 0.62246,-0.8859 0.41675,-0.3274 1.09058,-0.5954 0.85157,-1.264 0.14945,-0.2015 0.35234,-0.3468 0.55291,-0.495 0.10029,-0.074 0.27173,-0.214 0.3639,-0.2969 z"/>\n    </g>\n    <g class="z972 departement map-element">\n      <path d="m 122.03458,1267.3324 c -1.2683,1.3777 -0.41207,0.2067 0,0 z m 82.77978,12.7285 c 1.89458,0.2507 0.005,-0.7523 0,0 z m -20.54865,-18.5875 c -0.4903,0.6792 -0.45015,-0.7376 0,0 z m -15.44593,-116.3046 c -0.6526,-0.01 0.56431,-0.4492 0,0 z m 20.57806,-4.2114 c -0.15441,0.4027 -0.61836,-0.4845 0,0 z m -25.13581,16.809 c 1.66908,-0.2477 -0.25934,1.0786 0,0 z m 21.89708,17.7603 c -0.0847,0.6016 -0.7715,-0.3051 0,0 z m -0.0744,4.0534 c -0.26294,0.6207 -0.35034,-0.6958 0,0 z m 0.31273,-50.6853 c -0.83904,0.4891 0.3397,-0.7536 0,0 z m 12.72892,47.536 c 0.2706,1.1203